# Import packages

In [37]:
import math, os, collections
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from evaluation import Evaluation
from db_connection.utils import get_conn, get_data_start_dt
from utils_load import load_w103, load_w106, load_cust, cust_process
from mlaas_tools.config_build import config_set

In [38]:
## Configure env
if not os.path.isfile('config.ini'):
    config_set()

In [39]:
## Load db connection
rawdata_conn = get_conn('edu')

read key file
login as edu-cytsao


In [6]:
rawdata_conn = get_conn('edu')
# interaction train w103
w103_df = load_w103('2019-01-31', rawdata_conn, span=18)
purchase_hist_dca = pd.DataFrame(w103_df[w103_df['deduct_cnt']!=0].groupby("cust_no")["wm_prod_code"].apply(lambda x: list(set(x.values.tolist()))))
print(purchase_hist_dca.head())

read key file
login as edu-cytsao
                                wm_prod_code
cust_no                                     
++1QLn8bCFzxWN8vQYBuRg==              [NN18]
++2WLG2lyrgp9jhLkCFEKg==  [WW06, AG74, 2509]
++3bvnatF8Ie6ZC/bs3rcA==  [6216, 1204, 6017]
++7FzPdZNv2Gw99EJ/ZUUg==              [CC03]
++GMXRROXNCZtHYs3ymfyg==  [AG48, AG24, AG30]


In [7]:
rawdata_conn = get_conn('edu')
purchase_hist = w103_df.groupby("cust_no")["wm_prod_code"].apply(lambda x: list(set(x.values.tolist()))).to_dict()
evaluation = Evaluation('2019-01-31', None, '1m', purchase_hist)
evaluate_w103 = evaluation.read('2019-01-31', rawdata_conn, '1m')
evaluate_w103

read key file
login as edu-cytsao
read key file
login as edu-cytsao


,cust_no,wm_prod_code
0,00FmcV87beY8s7OI0Xjtkg==,5704
1,00IrxuhcWjiKQPXJDBv2Yg==,AG80
2,00IrxuhcWjiKQPXJDBv2Yg==,AG26
3,00IrxuhcWjiKQPXJDBv2Yg==,AC18
4,00roz6dL8l5Ouk54dlOc9A==,FC85
...,...,...
5664,zzjZadLsQ1PRT0/+2zULnA==,UF57
5665,zzOnaWQ8v9wgoMPXdIC63A==,BB01
5666,zzOnaWQ8v9wgoMPXdIC63A==,3706
5667,zzOnaWQ8v9wgoMPXdIC63A==,CCB7


In [8]:
evaluate_w103_hist = pd.DataFrame(evaluate_w103.groupby("cust_no")["wm_prod_code"].apply(lambda x: list(set(x.values.tolist()))))
evaluate_w103_hist

,wm_prod_code
cust_no,
+0pDrcJE0fvDekr6JLFYVQ==,[EE29]
+1Oc52w6Z5QfJwL+8BQ0Ew==,"[8915, ED64, FE41]"
+1y5TbuipnX4nqzoKTbQCw==,[AG80]
+2nQW/qhjI4QlLqIh+3GIg==,[1338]
+3725e06hHfZnvl65bmRSg==,[UF58]
...,...
zy3vgn+M6LkwhWL0o3x82A==,[AF51]
zyOOIBQoa3exIwHf7YKzIA==,[AV03]
zyPzzDNfmL3JASvX2a5Thg==,"[UF51, FE41]"


In [13]:
train_eva_hist = purchase_hist_dca.merge(evaluate_w103_hist, how='inner', left_index=True, right_index=True)
train_eva_hist

,wm_prod_code_x,wm_prod_code_y
cust_no,,
+0pDrcJE0fvDekr6JLFYVQ==,"[CC74, CCG3, PP56, NN03, CC23, AG25, AA91, XX0...",[EE29]
+1y5TbuipnX4nqzoKTbQCw==,"[CC88, AA55, AG69, AV03, AG25, AG21]",[AG80]
+3725e06hHfZnvl65bmRSg==,[BB51],[UF58]
+984UCbOEorzWUOaGOaBfQ==,"[AA39, KK32]",[UF34]
+9W7hR/cahy4WlpB7UOgmw==,"[AA77, 2018]",[AA05]
...,...,...
zuiSmmDqX+FNbpl+n5YmLg==,"[SS09, AW01, KK32]",[CCE2]
zy3vgn+M6LkwhWL0o3x82A==,"[JJ05, KK32]",[AF51]
zyOOIBQoa3exIwHf7YKzIA==,[AH11],[AV03]


In [14]:
train_eva_hist.loc[train_eva_hist['wm_prod_code_y'].isnull() == False]

,wm_prod_code_x,wm_prod_code_y
cust_no,,
+0pDrcJE0fvDekr6JLFYVQ==,"[CC74, CCG3, PP56, NN03, CC23, AG25, AA91, XX0...",[EE29]
+1y5TbuipnX4nqzoKTbQCw==,"[CC88, AA55, AG69, AV03, AG25, AG21]",[AG80]
+3725e06hHfZnvl65bmRSg==,[BB51],[UF58]
+984UCbOEorzWUOaGOaBfQ==,"[AA39, KK32]",[UF34]
+9W7hR/cahy4WlpB7UOgmw==,"[AA77, 2018]",[AA05]
...,...,...
zuiSmmDqX+FNbpl+n5YmLg==,"[SS09, AW01, KK32]",[CCE2]
zy3vgn+M6LkwhWL0o3x82A==,"[JJ05, KK32]",[AF51]
zyOOIBQoa3exIwHf7YKzIA==,[AH11],[AV03]


In [19]:
train_eva_hist = train_eva_hist.dropna()
dca_buy_again = []
dca_buy_again_fund = []
for i in range(len(train_eva_hist)):
    if type(train_eva_hist.iloc[i]['wm_prod_code_x']) != list or type(train_eva_hist.iloc[i]['wm_prod_code_y']) != list:
        dca_buy_again += ['NA']
    elif len(set(train_eva_hist.iloc[i]['wm_prod_code_x']) & set(train_eva_hist.iloc[i]['wm_prod_code_y'])) > 0:
        dca_buy_again += ['y']
    else:
        dca_buy_again += ['n']
    dca_buy_again_fund.append(list(set(train_eva_hist.iloc[i]['wm_prod_code_x']) & set(train_eva_hist.iloc[i]['wm_prod_code_y'])))
    
train_eva_hist['dca_buy_again'] = dca_buy_again
train_eva_hist['dca_buy_again_fund'] = dca_buy_again_fund

In [22]:
train_eva_hist[train_eva_hist['dca_buy_again']=='y']

,wm_prod_code_x,wm_prod_code_y,dca_buy_again,dca_buy_again_fund
cust_no,,,,
+ABoMPDclH9sdmFKkvHTwQ==,[KK41],[KK41],y,[KK41]
+LcBXj1EFcZfmE2fFiSOeA==,[CC79],[CC79],y,[CC79]
+VfDtNP136dC9fJ/HjpR0Q==,"[NN83, II33, NN23, NN68, CC87, BB51]",[CC87],y,[CC87]
+wDHmG+i5jRbbBj1yZ9xAw==,"[NN84, 2143, QQ49, NN83]",[NN83],y,[NN83]
16r0LJheBSV50oGJIW2ZFw==,"[AG23, AG24, 2128, AG20, AR07, AA08, CC55, 270...",[AG23],y,[AG23]
...,...,...,...,...
y4gVUcMUoGm08YOWVSDYwA==,"[BB16, OB08, PP65, KK08]",[KK08],y,[KK08]
yYu0lAMplnq+QudtpD91rQ==,"[AGE4, BB39]",[BB39],y,[BB39]
z0OVVSdUUHBB2O3FsmvGwQ==,"[KK68, KK09, NN82, BB51, KK41, BB39]",[BB39],y,[BB39]


In [23]:
train_eva_hist['dca_buy_again_fund'].explode('dca_buy_again_fund').value_counts()

BB39    13
AF58    12
KK41    11
BB51    11
NN18     8
        ..
2922     1
JJ05     1
AI19     1
2018     1
BB01     1
Name: dca_buy_again_fund, Length: 102, dtype: int64

In [119]:
'''
evaluate total/evaluate dca(在training時買過定期定額)/duplicate(又買之前申購過dca的基金)
2018-12-31 18m 3883/1454/203
2019-01-31 18m4101/1544/184
'''

'\nevaluate total/evaluate dca/duplicate\n2018-12-31 18m 3883/1454/203\n'

In [42]:
def check(date, s):
    rawdata_conn = get_conn('edu')
    # interaction train w103
    w103_df = load_w103(date, rawdata_conn, s)
    # user
    cust_df = load_cust(date, rawdata_conn, s)
    print('Total users:', len(w103_df.cust_no.unique()))
    # intersection
    user_filter = set(w103_df['cust_no'].tolist()) & set(cust_df['cust_no'].tolist())
    w103_df = w103_df[w103_df['cust_no'].isin(user_filter)]
    cust_df_filter = cust_df[cust_df['cust_no'].isin(user_filter)]
    print('Total users (with features):', len(w103_df.cust_no.unique()))
    
    purchase_hist = w103_df.groupby("cust_no")["wm_prod_code"].apply(lambda x: list(set(x.values.tolist()))).to_dict()
    purchase_hist_dca = pd.DataFrame(w103_df[w103_df['deduct_cnt']!=0].groupby("cust_no")["wm_prod_code"].apply(lambda x: list(set(x.values.tolist()))))

    # evaluation data
    evaluation = Evaluation(date, None, '1m', purchase_hist)
    evaluate_w103 = evaluation.read(date, rawdata_conn, '1m')
    # evaluation user features
    eva_cust_df = load_cust(date, rawdata_conn, span=None, mode='evaluation')
    # intersection
    eva_user_filter = set(evaluate_w103['cust_no'].tolist()) & set(eva_cust_df['cust_no'].tolist())
    eva_cust_df_filter = eva_cust_df[eva_cust_df['cust_no'].isin(eva_user_filter)]
    evaluate_w103 = evaluate_w103[evaluate_w103['cust_no'].isin(eva_user_filter)]
    # keep only warm start users
    warm_users, cold_users = evaluation.warm_cold_list()
    evaluate_w103 = evaluate_w103[evaluate_w103['cust_no'].isin(warm_users)]
    
    evaluate_w103_hist = pd.DataFrame(evaluate_w103.groupby("cust_no")["wm_prod_code"].apply(lambda x: list(set(x.values.tolist()))))
    
    train_eva_hist = purchase_hist_dca.merge(evaluate_w103_hist, how='inner', left_index=True, right_index=True)
    train_eva_hist = train_eva_hist.dropna()
    dca_buy_again, dca_buy_again_fund = [], []
    for i in range(len(train_eva_hist)):
        if type(train_eva_hist.iloc[i]['wm_prod_code_x']) != list or type(train_eva_hist.iloc[i]['wm_prod_code_y']) != list:
            dca_buy_again += ['NA']
        elif len(set(train_eva_hist.iloc[i]['wm_prod_code_x']) & set(train_eva_hist.iloc[i]['wm_prod_code_y'])) > 0:
            dca_buy_again += ['y']
        else:
            dca_buy_again += ['n']
        dca_buy_again_fund.append(list(set(train_eva_hist.iloc[i]['wm_prod_code_x']) & set(train_eva_hist.iloc[i]['wm_prod_code_y'])))
        
    train_eva_hist['dca_buy_again'] = dca_buy_again
    train_eva_hist['dca_buy_again_fund'] = dca_buy_again_fund
    
    print(date, str(s), 'evaluation total:', len(evaluate_w103_hist), 'evaluate dca:', len(train_eva_hist), 'duplicate:', len(train_eva_hist[train_eva_hist['dca_buy_again']=='y']))
    print(train_eva_hist['dca_buy_again_fund'].explode('dca_buy_again_fund').value_counts()[:5])

In [44]:
exp_dates = ['2019-01-31', '2019-02-28', '2019-03-31', '2019-04-30', '2019-05-31', '2019-06-30'] #'2018-12-31'
span = [18, 6]
for d in exp_dates:
    for s in span:
        check(d, s)

read key file
login as edu-cytsao
loading 2017-08-01 2019-01-31 data.
Total users: 72389
Total users (with features): 71759
read key file
login as edu-cytsao
loading 2019-02-01 2019-02-28 data.
2019-01-31 18 evaluation total: 3192 evaluate dca: 1532 duplicate: 183
BB39    13
AF58    12
KK41    11
BB51    11
NN18     8
Name: dca_buy_again_fund, dtype: int64
read key file
login as edu-cytsao
loading 2018-08-01 2019-01-31 data.
Total users: 49862
Total users (with features): 49498
read key file
login as edu-cytsao
loading 2019-02-01 2019-02-28 data.
2019-01-31 6 evaluation total: 2633 evaluate dca: 1388 duplicate: 144
AF58    10
KK41     9
BB51     9
BB39     9
AG23     5
Name: dca_buy_again_fund, dtype: int64
read key file
login as edu-cytsao
loading 2017-09-01 2019-02-28 data.
Total users: 71893
Total users (with features): 71271
read key file
login as edu-cytsao
loading 2019-03-01 2019-03-31 data.
2019-02-28 18 evaluation total: 5534 evaluate dca: 2559 duplicate: 268
KK41    24
AF58   

In [ ]:
# conclusion: 存在加碼申購定期定額的user，無法保證推薦中拿掉已經申購過的基金會提升成績